## CV Split

In [ ]:
# !pip3 install multiprocesspandas

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold
from multiprocesspandas import applyparallel
from tqdm import tqdm
from rank_bm25 import BM25Okapi, BM25L
from transformers import AutoTokenizer

ModuleNotFoundError: No module named 'rank_bm25'

In [4]:
input_path = r'../input_dir/'
topic_df = pd.read_csv(input_path + r'topics.csv')
content_df = pd.read_csv(input_path + r'content.csv')
corr_df = pd.read_csv(input_path + r'correlations.csv')
topic_df = topic_df.rename(columns={'id': 'topic_id'}).merge(corr_df)

In [5]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')

In [6]:
corr_df['content_ids'] = corr_df['content_ids'].apply(lambda x:x.split())
corr_df = corr_df.explode('content_ids').reset_index(drop=True)

In [7]:
topic_df = topic_df.fillna('')
topic_df['topic_full_text'] =  topic_df['title'] + ' [SEP] ' + topic_df['description']
topic_df = topic_df[['topic_id', 'topic_full_text', 'language', 'title']]
df = corr_df.merge(topic_df, left_on='topic_id', right_on='topic_id', how='left')
df = df[['topic_id','content_ids','topic_full_text','language', 'title']]
df = df.rename(columns={'language':'topic_language', 'title':'topic_title'})

In [8]:
content_df = content_df.fillna('')
content_df['content_full_text'] =  content_df['title'] + ' [SEP] ' + content_df['description'] + ' [SEP] ' + content_df['text']
content_df = content_df[['id', 'content_full_text', 'language', 'title']]
df = df.merge(content_df, left_on='content_ids', right_on='id', how='left')
df = df.rename(columns={'language':'content_language', 'title': 'content_title'})
df['label'] = 1

In [9]:
df.head()

,topic_id,content_ids,topic_full_text,topic_language,topic_title,id,content_full_text,content_language,content_title,label
0,t_00004da3a1b2,c_1108dd0c7a5d,Откриването на резисторите [SEP] Изследване на...,bg,Откриването на резисторите,c_1108dd0c7a5d,Молив като резистор [SEP] Моливът причинява пр...,bg,Молив като резистор,1
1,t_00004da3a1b2,c_376c5a8eb028,Откриването на резисторите [SEP] Изследване на...,bg,Откриването на резисторите,c_376c5a8eb028,Да чуем променливото съпротивление [SEP] Тук ч...,bg,Да чуем променливото съпротивление,1
2,t_00004da3a1b2,c_5bc0e1e2cba0,Откриването на резисторите [SEP] Изследване на...,bg,Откриването на резисторите,c_5bc0e1e2cba0,Променлив резистор (реостат) с графит от молив...,bg,Променлив резистор (реостат) с графит от молив,1
3,t_00004da3a1b2,c_76231f9d0b5e,Откриването на резисторите [SEP] Изследване на...,bg,Откриването на резисторите,c_76231f9d0b5e,Последователно свързване на галваничен елемент...,bg,Последователно свързване на галваничен елемент...,1
4,t_00068291e9a4,c_639ea2ef9c95,Entradas e saídas de uma função [SEP] Entenda ...,pt,Entradas e saídas de uma função,c_639ea2ef9c95,Dados e resultados de funções: gráficos [SEP] ...,pt,Dados e resultados de funções: gráficos,1


## bm2.5 or tfidf 采样 较难

In [10]:
def get_bm25_negative(sentence, candidates_sentence, thres=2):
    tokenized_corpus = [tokenizer.tokenize(i) for i in candidates_sentence]
    bm25 = BM25L(tokenized_corpus)
    tokenized_query = tokenizer.tokenize(sentence)
    doc_scores = bm25.get_scores(tokenized_query)
    index = np.argpartition(doc_scores, -2)[-2:]
    return np.array(candidates_sentence)[index].tolist()

In [11]:
%%time

#test_df = df[df['topic_id']=='t_000d1fb3f2f5']
neg_df = []
for topic_id in tqdm(df['topic_id'].unique()):
    sub_df = df[df['topic_id'] == topic_id]
    topic_language = sub_df['topic_language'].unique()[0]
    topic_full_text = sub_df['topic_full_text'].unique()[0]
    ## bm25
    bm25 = []
    
    querys = sub_df['content_full_text'].to_list()
    for i in querys:
        candidates = df[df['content_language'] == topic_language].sample(n=30)
        candidates_sentences = candidates['content_full_text'].to_list()
        bm25_negative = pd.DataFrame({'topic_full_text':[topic_full_text],
                                      'content_full_text':['']
                                     })
        results = get_bm25_negative(i, candidates_sentences)
        bm25_negative['content_full_text'] = '\u00001'.join(results)
        bm25_negative['topic_id'] = topic_id
        bm25_negative['label'] = 0
        bm25_negative['content_full_text'] = bm25_negative['content_full_text'].apply(lambda x:x.split('\u00001'))
        bm25_negative = bm25_negative.explode('content_full_text').reset_index(drop=True)
        bm25.append(bm25_negative)
    
    bm25 = pd.concat(bm25)
    bm25 = bm25[-(bm25['content_full_text'].isin(sub_df['content_full_text'].to_list()))]

    neg_df.append(bm25)


  1%|          | 414/61517 [09:08<22:28:12,  1.32s/it] 


KeyboardInterrupt: 

In [ ]:
output_path = r'/root/kaggle/output_dir/'
neg_df = pd.concat(neg_df)
neg_df = neg_df.drop_duplicates()
neg_df.to_csv(output_path + r'bm25_negative.csv', index=False)

In [17]:
neg_df.shape

In [18]:
df.shape

(102629, 9)